## Install dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install -e ../aicacia/extraction aicacia-document-exporter==0.1.4

## Extract from TEI files

In [ ]:
!cp ../aicacia/extraction/example/grobid/sample.grobid.tei.xml ../data/grobid

In [ ]:
from aicacia_extraction.grobid import TEIDocument

In [ ]:
tei_document = TEIDocument("../data/grobid/sample.grobid.tei.xml")

In [ ]:
tei_document.title

'Bi-criteria Algorithm for Scheduling Jobs on Cluster Platforms'

In [ ]:
tei_document.extract_text()[0:2]

['INTRODUCTION 1.1 Cluster computing',
 'The last few years have been characterized by huge technological changes in the area of parallel and distributed computing. Today, powerful machines are available at low price everywhere in the world. The main visible line of such changes is the large spreading of clusters which consist in a collection of tens or hundreds of standard almost identical processors connected together by a high speed interconnection network [6]. The next natural step is the extension to local sets of clusters or to geographically distant grids [10].']

In [ ]:
tei_document.extract_figure_descriptions()[0:2]

['Figure 1: Job submission in clusters.',
 'Figure 2: Principle of the algorithm.']